# 🚀 Projeto: Churn Prediction (SaaS 2025)
**Engenharia de Dados e Machine Learning no Databricks**

Este notebook executa o pipeline completo da camada **Silver** para a **Gold**, treinando um modelo de Random Forest para prever a probabilidade de cancelamento de assinaturas baseado no comportamento do usuário e no sentimento extraído (VoC).

## 1. Setup e Ingestão (Silver Layer)
Carregamos os dados brutos e realizamos a tipagem correta das colunas.

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

# Caminho definido para o Volume do Databricks
path_silver = "/Volumes/workspace/voc/churn/churn_silver_2025.csv"

df_silver = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(path_silver)

display(df_silver.limit(5))

## 2. Feature Engineering (Gold Layer)
Aqui transformamos dados brutos em inteligência de negócio:
- **Uso relativo:** Eficiência do uso do app em relação ao valor pago.
- **Tempo de Contrato:** Quantidade de dias desde a assinatura até a data de hoje.

In [ ]:
df_gold = df_silver.withColumn("data_assinatura", F.to_date("data_assinatura")) \
    .withColumn("uso_por_valor", F.col("total_logs_app_30d") / F.col("valor_mensalidade")) \
    .withColumn("dias_ativo", F.datediff(F.current_date(), F.col("data_assinatura"))) \
    .fillna(0)

print("Camada Gold gerada com sucesso!")
display(df_gold.select("id_cliente", "uso_por_valor", "dias_ativo", "churn").limit(5))

## 3. Preparação dos Dados para Machine Learning
Convertemos categorias de texto em números e agrupamos as features em um vetor denso.

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline

# Indexando categorias de texto
indexer = StringIndexer(inputCol="categoria_principal_voc", outputCol="categoria_index")

# Definindo as colunas preditoras
feature_cols = [
    "valor_mensalidade", 
    "total_logs_app_30d", 
    "tickets_suporte_abertos", 
    "score_sentimento_voc", 
    "uso_por_valor", 
    "dias_ativo", 
    "categoria_index"
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

## 4. Treinamento do Modelo (Random Forest)
Dividimos os dados (80% treino / 20% teste) e aplicamos o algoritmo.

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

train_data, test_data = df_gold.randomSplit([0.8, 0.2], seed=42)

rf = RandomForestClassifier(labelCol="churn", featuresCol="features", numTrees=100)

# Criando o Pipeline completo
pipeline = Pipeline(stages=[indexer, assembler, rf])

model = pipeline.fit(train_data)
predictions = model.transform(test_data)

print("Modelo treinado com sucesso!")

## 5. Avaliação de Performance
Analisamos a capacidade do modelo de distinguir entre clientes que ficam e que saem.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

eval_auc = BinaryClassificationEvaluator(labelCol="churn", metricName="areaUnderROC")
auc = eval_auc.evaluate(predictions)

eval_acc = MulticlassClassificationEvaluator(labelCol="churn", metricName="accuracy")
accuracy = eval_acc.evaluate(predictions)

print(f"Resultados: AUC-ROC: {auc:.4f} | Accuracy: {accuracy:.4f}")

## 6. Exportação para o Looker (Gold Layer Final)
Extraímos a probabilidade (que vem no vetor `probability` do Spark) para uma coluna legível.

In [ ]:
from pyspark.ml.functions import vector_to_array

churn_predictions_gold = predictions.withColumn("probabilidade_churn", vector_to_array("probability")[1]) \
    .select("id_cliente", "churn", "prediction", "probabilidade_churn", "score_sentimento_voc")

# Salvando como Delta Table (Ideal para o Looker ler via Databricks SQL)
# churn_predictions_gold.write.format("delta").mode("overwrite").saveAsTable("gold_churn_predictions")

display(churn_predictions_gold.orderBy(F.desc("probabilidade_churn")))